In [19]:
using Revise

In [20]:
using LearningMPC
using LearningMPC.Models
using RigidBodyDynamics
using RigidBodyDynamics.Contact
using MeshCatMechanisms
using RigidBodySim
using RigidBodySim.Visualization.MeshCatInterface
import LCPSim
using JLD2
using Blink

In [47]:
robot = BoxAtlas();

In [48]:
mvis = MechanismVisualizer(robot)
open(mvis, Window())

Blink.AtomShell.Window(5, Blink.AtomShell.Electron(Process(`/home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/Blink/deps/atom/electron /home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/Blink/src/AtomShell/main.js port 4767`, ProcessRunning), TCPSocket(RawFD(68) active, 0 bytes waiting), Dict{String,Any}(Pair{String,Any}("callback", Blink.#3))), Blink.Page(7, WebSockets.WebSocket(23, TCPSocket(RawFD(56) active, 0 bytes waiting), CONNECTED::WebSockets.ReadyState = 1), Dict{String,Any}(Pair{String,Any}("webio", WebIO.#109),Pair{String,Any}("callback", Blink.#3)), Future(1, 1, 7, Nullable{Any}(true))))

In [49]:
rootframe = root_frame(mechanism(robot))
floor = HalfSpace3D(Point3D(rootframe, 0., 0., 0.), FreeVector3D(rootframe, 0., 0., 1.))
add_environment_primitive!(mechanism(robot), floor)
wall = HalfSpace3D(Point3D(rootframe, 0., 1.0, 0.), FreeVector3D(rootframe, 0., -1., 0.))
add_environment_primitive!(mechanism(robot), wall)

feet = Dict(:left => findbody(mechanism(robot), "l_foot_sole"), :right => findbody(mechanism(robot), "r_foot_sole"))
hands = Dict(:left => findbody(mechanism(robot), "l_hand_mount"), :right => findbody(mechanism(robot), "r_hand_mount"));

contact_model = SoftContactModel(hunt_crossley_hertz(k = 500e3), ViscoelasticCoulombModel(0.8, 20e3, 100.))

for (side, body) in feet
    add_contact_point!(body, 
        ContactPoint(Point3D(default_frame(body), 0., 0, 0), 
                     contact_model))
end

for (side, body) in hands
    add_contact_point!(body, 
        ContactPoint(Point3D(default_frame(body), 0., 0, 0), 
                     contact_model))
end

In [50]:
@load "../2018-05-08-box-atlas-longer-miqp/boxatlas.jld2" net lqrsol mpc_params
mpc_params_data = mpc_params
mpc_params = MPCParams(robot)
mpc_params.Δt = mpc_params_data["Δt"]
mpc_params.horizon = mpc_params_data["horizon"]
net_cost = LearningMPC.LearnedCost(lqrsol, net)
net_mpc_params = MPCParams(robot)
net_mpc_params.horizon = 1
net_mpc_controller = MPCController(robot, net_mpc_params, net_cost, [lqrsol]);
full_mpc_controller = MPCController(robot, mpc_params, lqrsol, [lqrsol, net_mpc_controller]);
lqr_mpc_controller = MPCController(robot, (p = MPCParams(robot); p.horizon=1; p), lqrsol, [lqrsol])

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only


(::MPCController) (generic function with 1 method)

In [51]:
function inplace(controller)
    function (τ, t, state)
        τ .= controller(state)
    end
end

inplace (generic function with 1 method)

In [52]:
effort_bounds = LCPSim.all_effort_bounds(mechanism(robot))

function effort_limiter(τ::AbstractVector, t::Number, state::MechanismState)
    τ .= clamp.(τ, effort_bounds)
    τ
end

position_bounds = LCPSim.all_configuration_bounds(mechanism(robot))

function bounds_enforcer(τ::AbstractVector, t::Number, state::MechanismState)
    # TODO: handle q̇ vs v correctly
    for i in 1:num_positions(state)
        kp = 1000
        kd = 0.1 * kp
        if configuration(state)[i] > position_bounds[i].upper
            violation = configuration(state)[i] - position_bounds[i].upper
            τ[i] -= kp * violation
            τ[i] -= kd * velocity(state)[i]
        elseif configuration(state)[i] < position_bounds[i].lower
            violation = position_bounds[i].lower - configuration(state)[i]
            τ[i] += kp * violation 
            τ[i] -= kd * velocity(state)[i]
        end
    end
    τ
end

function compose(controllers...)
    function (τ, t, state)
        for c in controllers
            c(τ, t, state)
        end
        τ
    end
end

    

compose (generic function with 1 method)

In [53]:
num_additional_states(nominal_state(robot))

24

In [60]:
state = nominal_state(robot)
set_velocity!(state, findjoint(mechanism(robot), "floating_base"), [0.5, 0, 0.])
final_time = 5.
Δt = 0.01

composed = PeriodicController(similar(velocity(state)), Δt, compose(inplace(net_mpc_controller), effort_limiter))
controlcallback = DiffEqCallbacks.PeriodicCallback(composed)
composed = compose(composed, bounds_enforcer)

dynamics = Dynamics(mechanism(robot), composed)
problem = ODEProblem(dynamics, state, (0., final_time), callback = CallbackSet(controlcallback, CallbackSet(mvis, state, max_fps = 30.)))

@time sol = RigidBodySim.solve(problem, Tsit5(), abs_tol = 1e-8, dt = Δt);

116.489876 seconds (31.27 M allocations: 4.710 GiB, 2.21% gc time)


In [64]:
RigidBodySim.animate(mvis, state, sol)